In [1]:
import pandas as pd

d1 = pd.read_csv('data_clean\student_profile_clean.csv')
d1['student_id'] = d1['student_id'].str.strip().str.upper()
d1['full_name'] = d1['full_name'].str.strip().str.replace('  ', ' ') .str.replace('  ', ' ').str.lower().str.title()
d1['class_cohort'] = d1['class_cohort'].str.strip().str.upper().replace(' ', '', regex= True)

d2 = pd.read_csv('data_clean\club_registration_clean.csv')
d2['join_count'] = d2['join_count'].replace({'two': 2, '-1': 1})
d2['join_count'] = d2['join_count'].astype('int64')
d2['club_id'] = d2['club_id'].str.strip().str.lower()
d2['club_id'] = d2['club_id'].replace({'clb_amnhac': 'clb am nhac',
                                       'clb_kn_mem': 'clb ky nang mem',
                                       'clb_the_thao': 'clb the thao',
                                       'clb_tn': 'clb tot nghiep'}).str.title()
d2['student_id'] = d2['student_id'].str.strip()

d3 = pd.read_csv('data_clean\event_attendance_clean.csv', index_col= 'event_id')
d3['attendance_status'] = d3['attendance_status'].replace({'Vắng': 'Absent', 'Vang': 'Absent', 'vang ': 'Absent',
                                                           'Có mặt': 'Present', 'co  mat': 'Present', 'Co mat': 'Present',
                                                           'Muon': 'Late', 'Muộn': 'Late'})
d3['student_id'] = d3['student_id'].str.strip().str.upper()
d3['event_name'] = d3['event_name'].str.strip()
d3['club_id'] = d3['club_id'].str.strip().str.lower().str.capitalize()
d3['club_id'] = d3['club_id'].replace({'Clb_amnhac': 'Clb am nhac',
                                       'Clb_kn_mem': 'Clb ky nang mem',
                                       'Clb_the_thao': 'Clb the thao'})

In [2]:
a1 = pd.merge(d1, d3, on= 'student_id', how= 'outer')
a2 = a1.merge(d2, on= 'student_id', how= 'outer')
a2 = a2.drop_duplicates(['student_id', 'club_id_x', 'club_id_y']).dropna()
p1 = a2.pivot(index= ['student_id', 'club_id_x', 'club_id_y'],columns= 'attendance_status', values= 'join_count')
#print(p1.to_string())

a3 = d1.merge(d2, on= 'student_id', how= 'outer').drop_duplicates(['class_cohort', 'student_id']).dropna()
print(a3.to_string())
p2 = a3.pivot(index= 'student_id', columns= 'class_cohort', values= 'join_count')
#print(p2.to_string())
print(p1.stack().to_string())
print(p1.unstack().to_string())
print(p2.stack().to_string())
print(p2.unstack().to_string())
print(d3['club_id'].value_counts().max())
print(d3['club_id'].value_counts().min())

pp = d3.merge(d1, on= 'student_id', how= 'outer')
present_count = pp.groupby(['club_id', 'class_cohort', 'attendance_status']).size().reset_index()
print(present_count[present_count['attendance_status'] == 'Present'].value_counts())

    student_id     full_name class_cohort          club_id  join_count
0       SV2000    Tran Thi B        K18A2  Clb_Tinh_Nguyen         0.0
3       SV2002   Hoang Van E        K19A2  Clb_Tinh_Nguyen         2.0
4       SV2003      Le Van C        K18A1  Clb_Ky_Nang_Mem         2.0
5       SV2004    Tran Thi B        K19A1  Clb_Ky_Nang_Mem         0.0
7       SV2005      Le Van C        K19A2     Clb The Thao         0.0
9       SV2006  Nguyen Van A        K17A1     Clb The Thao         3.0
12      SV2007    Tran Thi B        K18A2  Clb_Ky_Nang_Mem         0.0
13      SV2008    Tran Thi B        K18A2      Clb_Am_Nhac         2.0
14      SV2009  Nguyen Van A        K18A1      Clb_Am_Nhac         5.0
17      SV2010   Hoang Van E        K20A1      Clb_Am_Nhac         3.0
19      SV2011    Tran Thi B        K20A2  Clb_Ky_Nang_Mem         0.0
20      SV2012    Pham Thi D        K20A1  Clb_Tinh_Nguyen         0.0
21      SV2013  Nguyen Van A        K19A2  Clb_Tinh_Nguyen         2.0
22    